In [1]:
using GLMakie
using JLD

In [7]:
include("../../src/run_model.jl")

main (generic function with 1 method)

In [3]:
function get_metadata(file_name::String)
    c = jldopen(file_name, "r") do file
        return read(file, "metadata")
    end
end

get_metadata (generic function with 1 method)

In [4]:
function load_matrix(timestep::Int, file_name::String, var::String)
    data_name = var * "_$timestep"
    c = jldopen(file_name, "r") do file
        return read(file, data_name)
    end
end

load_matrix (generic function with 1 method)

In [11]:
H_1 = H_2 = 1.0*KM
# H_2 = 2.0*KM
beta = 2*10^-11
Lx = 4000.0*KM # 4000 km
Ly = 4000.0*KM # 2000 km
dt = 15.0*MINUTES # 30 minutes
T = 0.5YEAR  # Expect to wait 90 days before seeing things.
U = 2.0 # Forcing term of top level.
M = P = 128
dx = Lx / M
# P = Int(Ly / dx)
visc = 100.0 # Viscosity, 100m^2s^-1
r = 10^-7 # bottom friction scaler.
R_d = 40.0*KM # Deformation radious, ~40km. Using 60km for better numerics.

model = BaroclinicModel(H_1, H_2, beta, Lx, Ly, dt, T, U, M, P, dx, visc, r, R_d)

sim_name = "test_30"
file_name = "../../data/$sim_name.jld"

zeta, psi = run_model(model, file_name)

# expected_zeta_0 = (-1 / (4pi^2 * model.Lx^-2 + 32pi^2Ly^-2)) * psi[:,:,1,1]

4.0e6
4.0e6
[-31250.0, 0.0, 31250.0, 62500.0, 93750.0, 125000.0, 156250.0, 187500.0, 218750.0, 250000.0, 281250.0, 312500.0, 343750.0, 375000.0, 406250.0, 437500.0, 468750.0, 500000.0, 531250.0, 562500.0, 593750.0, 625000.0, 656250.0, 687500.0, 718750.0, 750000.0, 781250.0, 812500.0, 843750.0, 875000.0, 906250.0, 937500.0, 968750.0, 1.0e6, 1.03125e6, 1.0625e6, 1.09375e6, 1.125e6, 1.15625e6, 1.1875e6, 1.21875e6, 1.25e6, 1.28125e6, 1.3125e6, 1.34375e6, 1.375e6, 1.40625e6, 1.4375e6, 1.46875e6, 1.5e6, 1.53125e6, 1.5625e6, 1.59375e6, 1.625e6, 1.65625e6, 1.6875e6, 1.71875e6, 1.75e6, 1.78125e6, 1.8125e6, 1.84375e6, 1.875e6, 1.90625e6, 1.9375e6, 1.96875e6, 2.0e6, 2.03125e6, 2.0625e6, 2.09375e6, 2.125e6, 2.15625e6, 2.1875e6, 2.21875e6, 2.25e6, 2.28125e6, 2.3125e6, 2.34375e6, 2.375e6, 2.40625e6, 2.4375e6, 2.46875e6, 2.5e6, 2.53125e6, 2.5625e6, 2.59375e6, 2.625e6, 2.65625e6, 2.6875e6, 2.71875e6, 2.75e6, 2.78125e6, 2.8125e6, 2.84375e6, 2.875e6, 2.90625e6, 2.9375e6, 2.96875e6, 3.0e6, 3.03125e6, 3.0

([-0.1739243090852484 -0.12131250604897689 … -0.1739243090852484 -0.12131250604897689; -0.1864690073020422 -0.13006245472671504 … -0.1864690073020422 -0.13006245472671504; … ; -0.1739243090852484 -0.12131250604897689 … -0.1739243090852484 -0.12131250604897689; -0.1864690073020422 -0.13006245472671504 … -0.1864690073020422 -0.13006245472671504;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; -0.1746582552470455 -0.12182443476475512 … -0.1746582552470455 -0.12182443476475512; -0.18717789868508536 -0.1305569076910291 … -0.18717789868508536 -0.1305569076910291; … ; -0.1746582552470455 -0.12182443476475512 … -0.1746582552470455 -0.12182443476475512; -0.18717789868508536 -0.1305569076910291 … -0.18717789868508536 -0.1305569076910291;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; -0.17539080624654185 -0.12233539035241833 … -0.17539080624654185 -0.12233539035241833; -0.18788529489947434 -0.1310503177726239 … -0.1878852

In [12]:
function animation(file_name)
    metadata = get_metadata(file_name)
    println(metadata)

    T = floor(Int, metadata["T"])
    dt = metadata["dt"]
    sample_timestep = 2*metadata["sample_timestep"]
    sample_interval = metadata["sample_interval"]

    zeta = load_matrix(0, file_name, "zeta")
    psi = load_matrix(0, file_name, "psi")

    # shift_amounts = (256, 128)
    shift_amounts = (0, 0)

    zeta_top_shifted = circshift(zeta[:,:,1], shift_amounts)
    zeta_bottom_shifted = circshift(zeta[:,:,2], shift_amounts)

    zeta_top_plot = Observable(zeta_top_shifted)
    zeta_bottom_plot = Observable(zeta_bottom_shifted)

    psi_top_shifted = circshift(psi[:,:,1], shift_amounts)
    psi_bottom_shifted = circshift(psi[:,:,2], shift_amounts)

    psi_top_plot = Observable(psi_top_shifted)
    psi_bottom_plot = Observable(psi_bottom_shifted)

    # plot_time = Observable(0.0)

    fig = Figure()
    ga = fig[1, 1] = GridLayout()
    
    ax_top_left = Axis(ga[1, 1], title="Zeta Layer 1")
    ax_bottom_left = Axis(ga[2, 1], title="Zeta Layer 2")
    ax_top_right = Axis(ga[1, 2], title="Psi Layer 1")
    ax_bottom_right = Axis(ga[2, 2], title="Psi Layer 2")

    heatmap!(ax_top_left, zeta_top_plot)
    heatmap!(ax_bottom_left, zeta_bottom_plot)
    heatmap!(ax_top_right, psi_top_plot)
    heatmap!(ax_bottom_right, psi_bottom_plot)

    expected_zeta_0 = (-1 / (4pi^2 * model.Lx^-2 + 32pi^2Ly^-2)) * psi[:,:,1]

    # display(psi_top_shifted)

    display(zeta[:,:,1])
    display(expected_zeta_0)

    display(fig)

    # display(psi[:,:,1])
    # display(psi[:,:,2])

    fps = 30
    sleep_time = 1 / fps

    for (timestep, time) in enumerate(0:dt:T)
        if timestep % (sample_timestep) == 0
            zeta = load_matrix(Int(timestep), file_name, "zeta")
            psi = load_matrix(Int(timestep), file_name, "psi")

            # println("$file_name, $timestep, top | ", zeta[10,10,1])
            # println("$file_name, $timestep, bottom | ", zeta[10,10,2])

            zeta_top_shifted = circshift(zeta[:,:,1], shift_amounts)
            zeta_bottom_shifted = circshift(zeta[:,:,2], shift_amounts) 

            zeta_top_plot[] = zeta_top_shifted
            zeta_bottom_plot[] = zeta_bottom_shifted

            psi_top_shifted = circshift(psi[:,:,1], shift_amounts)
            psi_bottom_shifted = circshift(psi[:,:,2], shift_amounts) 

            psi_top_plot[] = psi_top_shifted
            psi_bottom_plot[] = psi_bottom_shifted

            display(fig)

            # Set the matrix to nothing so the memoery can be freed up later by the garbage collector.
            matrix = nothing
            
            # Force garbage collection to clean up the memory used by the old array.
            GC.gc()
            sleep(sleep_time)
        end
    end

    sleep(1)
end

animation(file_name)

130×130 Matrix{Float64}:
  0.0495312    0.0345481    0.0192323   …   0.0495312    0.0345481
  0.0343193    0.0239378    0.0133257       0.0343193    0.0239378
  0.0190248    0.0132698    0.00738706      0.0190248    0.0132698
  0.00368441   0.00256988   0.0014306       0.00368441   0.00256988
 -0.0116649   -0.00813625  -0.0045293      -0.0116649   -0.00813625
 -0.026986    -0.0188228   -0.0104783   …  -0.026986    -0.0188228
 -0.0422422   -0.029464    -0.016402       -0.0422422   -0.029464
 -0.0573965   -0.0400342   -0.0222863      -0.0573965   -0.0400342
 -0.0724126   -0.0505079   -0.0281168      -0.0724126   -0.0505079
 -0.0872543   -0.06086     -0.0338796      -0.0872543   -0.06086
  ⋮                                     ⋱               
  0.150668     0.105091     0.0585024       0.150668     0.105091
  0.137039     0.0955849    0.0532103       0.137039     0.0955849
  0.12308      0.0858483    0.0477901       0.12308      0.0858483
  0.108824     0.0759048    0.0422548       0.108

130×130 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  ⋮                             ⋮    ⋱         ⋮                      
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.

Dict{String, Real}("T" => 1.5768e7, "sample_timestep" => 96, "dt" => 900.0, "sample_interval" => 86400.0)
